<a href="https://colab.research.google.com/github/gitmystuff/INFO5737/blob/main/Server_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers fastapi uvicorn python-multipart pyngrok -q

In [ ]:
import logging
from fastapi import FastAPI, Request, HTTPException
import uvicorn
import torch
import nest_asyncio
from google.colab import userdata
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM

logging.basicConfig(level=logging.DEBUG)

# Load the Chatbot Model
model_name = "gpt2"
# model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create the FastAPI App
app = FastAPI()

# Define an Endpoint for Chatbot Interaction
@app.post("/chat/")
async def chat(request: Request):
    try:
        data = await request.json()
        user_input = data["message"]
        history = data.get("history", [])

        # Construct the input prompt
        prompt = f"You are a concise chatbot. User: {user_input} Chatbot Response: "
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        # Generate the response
        output = model.generate(input_ids, max_length=500, pad_token_id=tokenizer.eos_token_id)
        bot_response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Update history
        new_history = history + [{"user": user_input, "bot": bot_response}]

        logging.debug(f"User Input: {user_input}")
        logging.debug(f"Bot Response: {bot_response}")
        logging.debug(f"History: {new_history}")

        return {"response": bot_response, "history": new_history}

    except Exception as e:
        logging.exception("Exception in /chat/ endpoint:")
        raise HTTPException(status_code=500, detail="Internal server error")

# Run the FastAPI App and Expose it with Ngrok
if __name__ == "__main__":
    nest_asyncio.apply()

    authtoken = userdata.get("NGROK_KEY")

    if authtoken:
        ngrok.set_auth_token(authtoken)

        http_tunnel = ngrok.connect(8000)
        print("Public URL:", http_tunnel.public_url)

        uvicorn.run(app, host="0.0.0.0", port=8000)
    else:
        print("Error: ngrok authtoken not found. Please add it to Colab user secrets as 'NGROK_KEY'.")